In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
from tqdm import tqdm_notebook as tqdm
import time
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def ceo():
    url = str('https://browsejjeds.jnj.com/SearchRoute.do?f=64172163061461&stp=a&sb=12&st=0&cc=0000076065')
    df = pd.read_html(url)
    we=df[0]
    xe=we[['WWID','Title']]
    xe=xe[xe['Title']=='Chairman/CEO']
    er=str(xe['WWID']).split(' ')[4].split('\n')[0]
    return er
er=ceo()

In [ ]:
def cleanString(string):
    cleaner = re.compile('<.*?>')
    cleanedString = re.sub(cleaner, '', str(string)).replace('\n', '').replace('\t', '').replace('\r', '').replace('[', '').replace(']', '').replace('\xa0', ' ').replace('amp;', '').strip()
    return cleanedString

In [ ]:
def getSoup(wwid):
    baseURLPart1 = 'https://browsejjeds.jnj.com/detail.jsp?detail=more&u='
    baseURLPart2 = '&st=01234'
    url = baseURLPart1 + wwid + baseURLPart2
    header = {'User-Agent': 'Mozilla/5.0'}
    request = Request(url, headers=header)
    webPage = urlopen(request)
    soup = BeautifulSoup(webPage)
    return soup

In [ ]:
def getEmployeeData(supervisorURL, level):
    global jnjEmployeeDataset
    employeeSoup = getSoup(supervisorURL.split('|')[0]).find('table')
    supervisorWWID = supervisorURL.split('|')[1]
    currentWWID = supervisorURL.split('&o=')[0]
    if employeeSoup.has_attr('class'):
        if employeeSoup['class'][0] == 'objectDetail':
            employeeDictionary = {}
            reporteeList = []
            for tableRow in employeeSoup.findAll('tr'):
                attributeLabel = tableRow.find('td', {'class':'label'}).text
                if attributeLabel != 'Direct Reports' and attributeLabel != 'Delegates' and attributeLabel != 'Sponsored Partners':
                    employeeDictionary[attributeLabel] = tableRow.find('td', {'class':'attribute'}).text
                elif attributeLabel == 'Direct Reports' or attributeLabel == 'Sponsored Partners':
                    try:
                        for directReportee in tableRow.find('select').findAll('option'):
                            reporteeURL = directReportee['value'].split('?detail=less&u=')
                            if len(reporteeURL) > 1:
                                finalReporteeURL = reporteeURL[1].split('&st=01234')[0]
                                reporteeWWID = finalReporteeURL.split('&o=')[0]
                                if reporteeWWID != supervisorWWID:
                                    reporteeList.append(finalReporteeURL + '|' + currentWWID)
                    except AttributeError:
                        reporteeURL = tableRow.findAll('a')[-1]['href'].split('?detail=less&u=')[1].split('&st=01234')[0]
                        reporteeWWID = reporteeURL.split('&o=')[0]
                        reporteeList.append(reporteeURL + '|' + currentWWID)
            employeeDictionary['Supervisor WWID'] = supervisorWWID
            employeeDictionary['Level'] = level
            jnjEmployeeDataset = jnjEmployeeDataset.append(employeeDictionary, ignore_index=True, sort=False)
            return reporteeList
        elif employeeSoup['class'][0] == 'errortable':
            print('The searched employee does not exist!')
            print(supervisorWWID)
            return []
        else:
            print('Unknown error!')
            return []

In [ ]:
levelLimit = 4
retryInterval = 900
supervisorList = [er+'&o=employees|'+er]

#22500016
# supervisorList = supervisorList[20548:]

jnjEmployeeDataset = pd.DataFrame()

for level in range(0, levelLimit, 1):
    nextSupervisorList = []
    print('\rFetching C-' + str(level) + ' Employees... \t\t', end='')
    for supervisor in tqdm(supervisorList, leave=False):
        try:
            nextSupervisorList = nextSupervisorList + getEmployeeData(supervisor, level)
        except Exception as error:
            print(Exception)
#             for secondCounter in range(retryInterval, 0, -60):
#                 print('\rRetrying in', int(secondCounter/60), 'minute(s) \t\t', end='')
#                 time.sleep(60)
#             nextSupervisorList = nextSupervisorList + getEmployeeData(supervisor, level)
    supervisorList = nextSupervisorList
    clear_output()

In [ ]:
jnjEmployeeDataset.loc[jnjEmployeeDataset['Formatted Name'].isna(), 'Formatted Name'] = jnjEmployeeDataset['Common Name']

In [ ]:
jnjEmployeeDataset.to_excel('J&J Org Data (C-8 Levels).xlsx', index=False)